In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
import math
from numpy import *
from sklearn.svm import LinearSVC

from datetime import datetime

from skimage.color import rgb2gray
from skimage.feature import hog

DATA_PATH = "D:/学习/课程资料/计算机视觉/cifar-10-python/cifar-10-batches-py/"

def unpack(fname):
    import pickle
    with open(fname, "rb") as fin:
        dumped = pickle.load(fin, encoding="bytes")
    return dumped

def reshape(data):
    img = np.zeros((32, 32, 3), 'uint8')
    img[..., 0] = np.reshape(data[:1024], (32, 32))
    img[..., 1] = np.reshape(data[1024:2048], (32, 32))
    img[..., 2] = np.reshape(data[2048:3072], (32, 32))
    return img

N = 5
test_num = 10000
bestLoss = 100000
bestWeights = 0
loss = []

batches_meta = unpack(DATA_PATH + "batches.meta")
data_batches = [
    unpack(DATA_PATH + "data_batch_" + str(i+1))
    for i in range(5)
]
test_batch = unpack(DATA_PATH + "test_batch")

In [3]:
from skimage.color import rgb2gray
from skimage.feature import local_binary_pattern
from skimage.feature import hog
radius = 3
n_points = 8 * radius
data_lbp = []
labels_lbp = []
data_all = {}
for i in range(5):
    labels_lbp.extend(data_batches[i][b"labels"])
    for img in data_batches[i][b"data"]:
        tmp  = local_binary_pattern(rgb2gray(reshape(img)), n_points, radius)
        data_lbp.append(tmp.flatten())

data_all['labels'] = labels_lbp
data_all['data'] = data_lbp        

# indext = 0
test_lbp = []
test_all = {}
for img in test_batch[b"data"]:
        tmp  = local_binary_pattern(rgb2gray(reshape(img)), n_points, radius)
        test_lbp.append(tmp.flatten())

test_all['labels'] = test_batch[b"labels"]
test_all['data'] = test_lbp

In [4]:
W = np.random.random((10,1024))
W = mat(W)

In [5]:
#计算梯度
def evaluate_gradient(f, x, W):
  global bestWeights
  global bestLoss
  global loss
  xmatrics = mat(x["data"])
  matrics = W*xmatrics.T
  fx = f(matrics, x["labels"]) # evaluate function value at original point
  if (fx < bestLoss):
      bestWeights = W
      bestLoss = fx
  loss.append(fx)
#   ks = linspace(1,1000,1000)
#   plt.plot(ks, loss)
  print("loss is", fx)
  grad = np.zeros(W.shape)
  h = 0.0001
  [idx, idy] = W.shape

  for i in range(idx):
        for j in range(idy):
            # evaluate function at x+h
            ix = i
            iy = j
            old_value = W[ix, iy]
            W[ix, iy] = old_value + h # increment by h
            matrics = W*xmatrics.T
            fxh = f(matrics, x["labels"]) # evalute f(x + h)
            W[ix, iy] = old_value # restore to previous value (very important!)

            # compute the partial derivative
            grad[ix, iy] = (fxh - fx) / h # the slope
            # W += - 0.1 * mat(grad)
  return mat(grad)

#定义softmax 损失函数
def loss_fun(data, labelList):
    [x, y] = data.shape
    L = 0
    for i in range(0, y):
          sum = 0
          for j in range(0, x):
                sum += math.exp(data[j, i])
          L += -math.log10(math.exp(data[labelList[i], i]) / sum)
    return L

#进行图片采样
def pics_sample(data, num):
      res = {}
      labelList = []
      dataList = []
      for i in range(0, num):
            randomNum = random.randint(0, 10000 * N)
            randomData = data["data"][randomNum]
            randomLabel = data["labels"][randomNum]
            labelList.append(randomLabel)
            dataList.append(randomData)
      res['data'] = dataList
      res['labels'] = labelList
      return res

#设置步长
step_size = 0.2
i = 100
times = linspace(1,i,i)

while i > 0:
      # 图片采样
      data = pics_sample(data_all, 256)
      # 计算梯度
      weights_grad = evaluate_gradient(loss_fun, data, W)
      # 更新梯度
      W += - step_size * weights_grad  
      i = i - 1

pltloss = loss[20:120]

OverflowError: math range error

In [ ]:
print ("best loss", bestLoss)
plt.plot(times,pltloss)

In [ ]:
right_num = 0
test_matrix = mat(test_all["data"])
test_result = bestWeights * test_matrix.T
for j in range(10000):
      biggest_num = -10000
      result_label = 10
      for i in range(10):
            if (biggest_num < test_result[i, j]):
                  biggest_num = test_result[i, j]
                  result_label = i
      if (result_label == test_all["labels"][j]):
            right_num += 1
print("right rate", right_num/10000)